# Decision Tree ( ID3 Algorithm )

In [ ]:
import numpy as np
import pandas as pd
import pprint

In [ ]:
df = pd.read_csv('data.csv')
df

,Outlook,Temperature,Humidity,Wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [ ]:
# Constant Value To Avoid Divide By Zero Error
eps = np.finfo(float).eps

In [ ]:
# Calculating Entropy For Entire Dataset
def calculate_entropy(df):
  target_label = df.keys()[-1]
  entropy = 0
  # Target Categories ['yes', 'no']
  values = df[target_label].unique()
  # Calculate S-Value / Entropy
  for value in values:
    probability = df[target_label].value_counts()[value] / len(df[target_label])
    entropy += -( probability * np.log2(probability))
  return entropy

$$
\text{Entropy}(S) = - \sum_{i=1}^{c} p_i \log_2(p_i)
$$

In [ ]:
# Calculating Entropy With Respect To Single Feature
def calculate_entropy_feature(df, feature):
  target_label = df.keys()[-1]
  # This Will Give Us all Yes And No's
  target_categories = df[target_label].unique()

  # This gives us unique categories in feature
  # Eg. In Outlook We've 3 Categories : Sunny, Overcast And Rainy
  categories = df[feature].unique()
  weighted_entropy = 0

  # Calculate Entropy Over Each Category From Feature
  # And Calucate Weighted Entropy For That Feature
  for category in categories:
    entropy = 0

    for target_variable in target_categories:
      # numerator
      num = len(df[feature][df[feature]==category][df[target_label] == target_variable])
      # denominator
      den = len(df[feature][df[feature]==category])

      # Fraction of target_labels ["Yes / No"] in the sub table formed after knowing a cateogry of a feature
      probability = num/(den+eps)

      # eps To Avoid Divide By Zero Error
      entropy += -( probability * np.log2( probability + eps ) )

    # How much of the weight Each Category Hold in the feature
    weight = den/len(df)
    weighted_entropy += -( weight * entropy )
  # Return only absolute value of entropy
  return abs(weighted_entropy)

In [ ]:
def find_root(df):
  # df.keys()[:-1] , Will output list of all features except target
  features = df.keys()[:-1]
  information_gain = []

  # iterate over all the features
  # calculate information gain
  for key in features:
    print(key,calculate_entropy_feature(df,key))
    information_gain.append( calculate_entropy(df) - calculate_entropy_feature(df,key) )

  # return feature with highest information gain
  return df.keys()[:-1][np.argmax(information_gain)]


In [ ]:
find_root(df)

Outlook 0.6935361388961914
Temperature 0.9110633930116756
Humidity 0.7884504573082889
Wind 0.892158928262361


'Outlook'

In [ ]:
# When we want a portion of dataset based on a feature category
# Node : Feature Name , Value : Category In Feature
def get_subtable(df, node, value):
  return df[df[node] == value].reset_index(drop=True)
  # drop other entries

In [ ]:
def buildTree(df, tree = None):
  target_label = df.keys()[-1]
  # find root node based on information gain
  node = find_root(df)
  # collect categories from root node
  attValue = np.unique(df[node])
  # Create a empty dictonary to hold root node
  if tree is None:
    tree={}
    tree[node] = {}
  # for each category
  # find its subtable and repeat the process to build decision tree
  for value in attValue:
    subtable = get_subtable(df, node, value)
    # find all 0's and 1's etc from target label from the subtables along with there counts
    clValue,counts = np.unique(subtable[target_label],return_counts=True)
    # if there is no randomness in target_label then we'll get final target label
    if len(counts)==1:
      tree[node][value] = clValue[0]
    else:
      # build tree recursively if there there is still randomness in the target_label
      tree[node][value] = buildTree(subtable)
  return tree

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.head()

In [ ]:
tree = buildTree(df)

In [ ]:
pprint.pprint(tree)

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}


In [ ]:
# Test Model
def test_model(test_case, tree):
  # iter function create a iterator
  tree_iterator = iter(tree)
  # next function returns next item in iterator
  attribute = next(tree_iterator)
  print(attribute)

  if test_case[attribute] in tree[attribute].keys():
    print(tree[attribute].keys())
    print(test_case[attribute])
    # create a subtree after matching root node
    result = tree[attribute][test_case[attribute]]
    pprint.pprint(result)
    # is instance for type checking
    # if result is dictonary then we need to further go deeper in tree
    # else we have final result
    if isinstance(result, dict):
      # test again with the subtree
      return test_model(test_case, result)
    else:
      return result
  else:
    # if attribute does not match then return None
    # as test case has attribute not present in tree
    return None

In [ ]:
test_case = {'Outlook':'Sunny', 'Temperature':'Hot', 'Humidity':'High', 'Wind':'Weak'}

In [ ]:
result = test_model(test_case, tree)

Outlook
dict_keys(['Overcast', 'Rain', 'Sunny'])
Sunny
{'Humidity': {'High': 'No', 'Normal': 'Yes'}}
Humidity
dict_keys(['High', 'Normal'])
High
'No'


In [ ]:
result

'No'

### Task
1. Create Decision tree with already existing sklearn library and running above program and your progran on multiple datasets and check how results differ.
2. Modify entropy in current code to implement decision tree with other method eg. gini impurity and observe the changes.
3. Instead of taking only categorial values in target, try to work on dataset where target values can take continous values (i.e Regression Tree).